In [324]:
import pandas as pd
import numpy as np

## FPTP results

In [325]:
provs = ['BC', 'AB', 'SK', 'MB', 'ON', 'QC', 'NB', 'NS', 'PE', 'NL', 'YT', 'NT', 'NU']

In [327]:

df = pd.DataFrame.from_csv('../data/fptp/summary_results_table11.csv', index_col=None)

In [328]:
df['candidate_and_party'] = df['Elected Candidate/Candidat \xc3\xa9lu']
df['riding_name'] = df['Electoral District Name/Nom de circonscription']
df['riding_number'] = df['Electoral District Number/Num\xc3\xa9ro de circonscription']
df['prov'] = df[df.columns[0]]

In [330]:
df = df[['prov', 'candidate_and_party', 'riding_name', 'riding_number']].copy()

In [331]:
parties = {'Liberal/Lib\xc3\xa9ral': 'Liberal',
           'Conservative/Conservateur': 'Conservative',
           'NDP-New Democratic Party/NPD-Nouveau Parti d\xc3\xa9mocratique': 'NDP',
           'Bloc Qu\xc3\xa9b\xc3\xa9cois/Bloc Qu\xc3\xa9b\xc3\xa9cois': 'Bloc',
           'Green Party/Parti Vert': 'Green'}

In [332]:
df['party'] = pd.Series(index=df.index, dtype=str)

In [333]:
for i in df.index:
    for k in parties:
        if k in df['candidate_and_party'][i]:
            df.set_value(i, 'party', parties[k])
            

In [334]:
df[df.party != df.party]

,prov,candidate_and_party,riding_name,riding_number,party


In [335]:
del df['candidate_and_party']

In [336]:
prov_dict = {'Newfoundland and Labrador/Terre-Neuve-et-Labrador': 'NL',
             'Prince Edward Island/Île-du-Prince-Édouard': 'PE',
             'Nova Scotia/Nouvelle-Écosse': 'NS',
             'New Brunswick/Nouveau-Brunswick': 'NB',
             'British Columbia/Colombie-Britannique': 'BC',
             'Yukon': 'YT',
             'Northwest Territories/Territoires du Nord-Ouest': 'NT',
             'Nunavut': 'NU',
             'Alberta': 'AB',
             'Manitoba': 'MB',
             'Ontario': 'ON',
             'Quebec/Qu\xc3\xa9bec': 'QC',
             'Saskatchewan': 'SK'
            }

In [337]:
df['prov_abbr'] = df['prov'].map(prov_dict)

In [338]:
df[df.prov_abbr != df.prov_abbr]

,prov,riding_name,riding_number,party,prov_abbr


In [339]:
del df['prov']

In [343]:
df['region_number'] = df['prov_abbr'].map(lambda x: provs.index(x))

In [374]:
df['seat_type'] = ['local']*len(df)

In [375]:
df.to_csv('../web/data/fptp_clean.csv', index=None)

## DMP Simulation Results

In [346]:
dmp = pd.DataFrame.from_csv('../data/dmp/dmp_2_5_0_reformatted.csv', index_col=None)

In [347]:
prov_dict_2 = {'Newfoundland': 'NL',
             'Prince Edward Island': 'PE',
             'Nova Scotia': 'NS',
             'New Brunswick': 'NB',
             'British Columbia': 'BC',
             'Yukon': 'YT',
             'Northwest Territories': 'NT',
             'Nunavut': 'NU',
             'Alberta': 'AB',
             'Manitoba': 'MB',
             'Ontario': 'ON',
             'Quebec': 'QC',
             'Saskatchewan': 'SK'
            }

In [348]:
dmp['prov_abbr'] = dmp['prov'].map(prov_dict_2)

In [349]:
del dmp['prov']

In [350]:
dmp['riding_number'] = range(len(dmp))


In [351]:
# I need 1 row per seat
dmp_1 = dmp[['riding_name', 'riding_number', 'party_1', 'prov_abbr']].copy()
dmp_2 = dmp[['riding_name', 'riding_number', 'party_2', 'prov_abbr']].copy()
dmp_1['party'] = dmp_1['party_1']
dmp_2['party'] = dmp_2['party_2']
del dmp_1['party_1']
del dmp_2['party_2']

In [352]:
dmp_2.index = range(len(dmp_1), len(dmp_1) + len(dmp_2))

In [353]:
dmp_3 = pd.concat([dmp_1, dmp_2])

In [354]:
dmp_3.head()

,riding_name,riding_number,prov_abbr,party
0,Surrey,0,BC,Liberal
1,Vancouver Island South,1,BC,NDP
2,Fraser Canyon,2,BC,Liberal
3,Abbotsford Area,3,BC,Conservative
4,Boundary Bay,4,BC,Liberal


In [355]:
for i in dmp_3.index:
    if dmp_3['party'][i] == 'Bloc Quebecois':
        dmp_3.set_value(i, 'party', 'Bloc')

In [357]:
dmp_3['region_number'] = dmp_3['prov_abbr'].map(lambda x: provs.index(x))

In [372]:
dmp_3['seat_type'] = ['local']*len(dmp_3)

In [373]:
dmp_3.to_csv('../web/data/dmp_clean.csv', index=None)

## MMP Simulation Results

Wilf Day's blog: http://wilfday.blogspot.ca/

My simulation using this model lets voters elect 209 local MP in the ten provinces, and 126 regional MPs in 42 regions.

In Atlantic Canada each province is one region. The other regions are:

Ontario:
Ottawa—Cornwall 10, East Central Ontario (Kingston—Peterborough) 9, Durham—Rouge Park 6, York Region 10, Scarborough—Don Valley 8, Toronto Central 8, North York—Etobicoke 8, Brampton—Mississauga North 7, Mississauga—Halton 8, Hamilton—Niagara—Brant 11, Central Ontario (Barrie—Owen Sound) 6, Waterloo-Wellington-Dufferin 8, London-Oxford-Perth-Huron 7, Windsor—Sarnia 6, Northern Ontario 9.

Quebec:
Montreal-est 8, Montreal-ouest 6, Montreal-nord—Laval 8, Laurentides—Lanaudière 9, Outaouais—Abitibi-Témiscamingue—Nord 6, Longueuil—Suroît 10, Montérégie-est—Estrie 6, Mauricie—Centre-du-Québec 6, Quebec City—Saguenay-Lac-Saint-Jean—Côte-Nord 11, Chaudière-Appalaches—Bas-Saint-Laurent—Gaspésie 8.

Manitoba:
Winnipeg 8, Manitoba North and South 6

Saskatchewan:
Regina—South Saskatchewan 6, Saskatoon—North Saskatchewan 8

Alberta:
Calgary 10, South & Central Alberta 8, Edmonton 10, Northern Alberta 6

British Columbia:
Vancouver-Richmond-Delta 9, Burnaby—North Shore—Coquitlam—Maple Ridge 8, Surrey—Fraser Valley 9, BC Interior and North 9, Vancouver Island 7

Voters for all three parties would elect either local or regional MPs in each of the 42 regions across Canada, except no Conservative in 8-MP Montreal-est and in 6-MP Outaouais—Abitibi-Témiscamingue—Nord.  Bloc voters would elect MPs in each of Quebec’s regions except 6-MP Montreal West. 

The results for 338 MPs are: 145 Liberals, 106 Conservatives, 70 NDP, 14 Bloc, and 3 Greens.



Lib, Con, NDP, Green, Bloc
B.C.                15, 13, 11, 3
Alberta           8, 21, 4, 1       
Sask.              3, 7, 4
Manitoba       6, 5, 2, 1
Ontario           55, 43, 20, 3
Quebec          28, 13, 20, 2, 15
N.B.                5, 3, 2
P.E.I.               2, 1, 1
N.S.                7, 2, 2
Nfld & Lab.    5, 1, 1
Territories      3, 0, 0
Total               137, 109, 67, 10, 15

In [402]:
mmp = pd.DataFrame.from_csv('../data/mmp/mmp_14_reformatted.csv', index_col=None)

In [404]:
mmp = mmp.replace(to_replace=' ', value=np.nan)

In [405]:
mmp.head()

,Electoral District,prov_abbr,n_local,n_region,BQ,C,Gr,L,ND,In,BQ.1,C.1,Gr.1,L.1,NDP
0,Vancouver Island 7 (4+3),BC,4,3,NaN,NaN,1.0,NaN,3.0,NaN,NaN,1.0,1.0,1.0,NaN
1,BC Interior and North 9 (6+3),BC,6,3,NaN,3.0,NaN,1.0,2.0,NaN,NaN,NaN,NaN,2.0,1.0
2,Surrey—Richmond—Abbotsford—Langley 12 (7+5),BC,7,5,NaN,3.0,NaN,4.0,0.0,NaN,NaN,1.0,1.0,1.0,2.0
3,Vancouver—Burnaby—North Shore—Maple Ridge 14 (...,BC,9,5,NaN,NaN,NaN,6.0,3.0,NaN,NaN,3.0,1.0,NaN,1.0
4,Edmonton and North Alberta 16 (10+6),AB,10,6,NaN,8.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,3.0,2.0


In [406]:
mmp['region_number'] = range(len(mmp))

In [407]:
mmp_local = mmp[list(mmp.columns[4:9]) + ['region_number', 'prov_abbr']].copy()
mmp_regional = mmp[list(mmp.columns[10:]) + ['prov_abbr']].copy()

In [408]:
parties = ['Bloc', 'Conservative', 'Green', 'Liberal', 'NDP']

In [409]:
mmp_regional.columns = parties + ['region_number', 'prov_abbr']
mmp_local.columns = parties + ['region_number', 'prov_abbr']

In [410]:
mmp_local['seat_type'] = ['local']*len(mmp_local)
mmp_regional['seat_type'] = ['regional']*len(mmp_regional)

In [411]:
mmp_regional.index = range(len(mmp_local), len(mmp_local)+len(mmp_regional))
mmp2 = pd.concat([mmp_local, mmp_regional])

In [412]:
mmp3 = pd.DataFrame(columns=['region_number', 'prov_abbr', 'seat_type', 'party', 'riding_number'], index=range(338))
i2 = 0
n = 0
for i1 in mmp2.index:
    for party in parties:
        if mmp2[party][i1] > 0:
            for _ in range(int(mmp2[party][i1])):
                for col in ['region_number', 'prov_abbr', 'seat_type']:
                    mmp3.set_value(i2, col, mmp2[col][i1])
                if mmp2['seat_type'][i1] == 'local':
                    mmp3.set_value(i2, 'riding_number', n)
                    n += 1
                mmp3.set_value(i2, 'party', party)
                i2 += 1

In [413]:
mmp3.head()

,region_number,prov_abbr,seat_type,party,riding_number
0,0,BC,local,Green,0
1,0,BC,local,NDP,1
2,0,BC,local,NDP,2
3,0,BC,local,NDP,3
4,1,BC,local,Conservative,4


In [414]:
mmp3.to_csv('../web/data/mmp_clean.csv', index=None)

## Instant Runoff Voting (IRV/AV)

In [362]:
irv = pd.DataFrame.from_csv('../data/irv/irv_prov_grenier.csv', index_col=None)

In [363]:
irv

,prov_abbr,Liberal,Conservative,NDP,Green,Bloc
0,BC,26,2,13,1,0
1,AB,6,26,2,0,0
2,SK,2,8,4,0,0
3,MB,8,4,2,0,0
4,ON,96,16,9,0,0
5,QC,51,5,20,0,2
6,NB,10,0,0,0,0
7,NS,11,0,0,0,0
8,PE,4,0,0,0,0
9,NL,7,0,0,0,0


In [365]:
parties = ['Bloc', 'Conservative', 'Green', 'Liberal', 'NDP']
irv2 = pd.DataFrame(columns=['prov_abbr', 'party'], index=range(338))
i2 = 0
for i1 in irv.index:
    for party in parties:
        if irv[party][i1] > 0:
            for _ in range(int(irv[party][i1])):
                irv2.set_value(i2, 'prov_abbr', irv['prov_abbr'][i1])
                irv2.set_value(i2, 'party', party)
                i2 += 1

In [366]:
irv2['riding_number'] = irv2.index

In [368]:
irv2['region_number'] = irv2['prov_abbr'].map(lambda x: provs.index(x))

In [370]:
irv2['seat_type'] = ['local']*len(irv2)

In [371]:
irv2.to_csv('../web/data/irv_clean.csv', index=None)